In [1]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser

In [2]:
import pandas as pd
from pandas import Series, DataFrame

In [3]:
sen_docs = pd.read_csv("recent.csv")
sen_docs['id']=sen_docs['id'].astype(str)
sen_docs['user']=sen_docs['user'].astype(str)
sen_docs['time']=pd.to_datetime(sen_docs.time,unit='s')
sen_docs['hashtags'] = sen_docs.text.str.findall('(#[a-zA-Z]*\w)')
sen_docs['ats'] = sen_docs.text.str.findall('(@[a-zA-Z]*\w)')
sen_docs['url_domains'] = sen_docs.urls.str.findall('://([a-zA-Z]*\w)')

In [4]:
sen_docs.sample(5)

,Unnamed: 0,id,text,time,urls,user,hashtags,ats,url_domains
1196,1196,1042805144990228481,There isn’t one solution to fighting the #opio...,2018-09-20 15:58:15,https://twitter.com/i/web/status/1042805144990...,2293131060,[#opioidepidemic],[],[twitter]
258,258,1039973950829355009,"More good news → ""Middle-class income rose to ...",2018-09-12 20:28:06,https://twitter.com/i/web/status/1039973950829...,237862972,[],[],[twitter]
1178,1178,1045394307191132160,I am humbled by your bravery and patriotism. \...,2018-09-27 19:26:39,http://twitter.com/cnnbrk/status/1045322966580...,2293131060,"[#IBelieveChristineBlaseyFord, #KavanuaghHeari...",[],[twitter]
498,498,1053780681984217088,Trump's policy of separating children from the...,2018-10-20 22:51:07,https://twitter.com/i/web/status/1053780681984...,14135426,[],[],[twitter]
1051,1051,1036622426690469889,"Labor Day is more than a day off, it is about ...",2018-09-03 14:30:20,https://twitter.com/i/web/status/1036622426690...,1262814457,[],[],[twitter]


In [6]:
schema = Schema(
    content=TEXT(stored=True), 
    id=ID(stored=True),
    hashtags=KEYWORD,
    ats=KEYWORD,
    urls=TEXT
)
ix = create_in("indexdir", schema)

In [7]:
with ix.writer() as writer:
    for doc in sen_docs.iterrows():
        try:
            writer.add_document(
                content=doc[1].text, 
                id=str(doc[1].id),
                hashtags=','.join(doc[1].hashtags),
                ats=','.join(doc[1].ats),
                urls=' '.join(doc[1].urls)
            )
        except:
            pass

In [10]:
search_term = 'cnbc'

with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse(search_term)
    results = searcher.search(query)
    result_ids = []
    for r in results:
        result_ids.append(r['id'])
    results_df = sen_docs[sen_docs.id.isin(result_ids)]
results_df

,Unnamed: 0,id,text,time,urls,user,hashtags,ats,url_domains
175,175,1057711572812996609,"Via @CNBC: Wages and salaries jump by 3.1%, hi...",2018-10-31 19:11:04,https://www.cnbc.com/2018/10/31/wages-and-sala...,237862972,[],[@CNBC],[www]
205,205,1052263036109635591,Via @CNBC: Another great sign for the economy:...,2018-10-16 18:20:32,https://www.cnbc.com/2018/10/16/job-openings-h...,237862972,[],[@CNBC],[www]
238,238,1044666690712334337,Via @CNBC: Consumer confidence rises to highes...,2018-09-25 19:15:22,https://cnb.cx/2xPvvnv,237862972,[],[@CNBC],[cnb]
276,276,1034552480602963971,Via @CNBC: Consumer confidence pops in August ...,2018-08-28 21:25:07,https://www.cnbc.com/2018/08/28/august-consume...,237862972,[],[@CNBC],[www]
